In [2]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy, IndependentlyCenter
from neuralnilm.data.datathread import DataThread

In [34]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = SEQ_LENGTH
WINDOWS = {
    'train': {
        1: ("2014-01-01", "2014-02-01")
    },
    'unseen_activations_of_seen_appliances': {
        1: ("2014-02-02", "2014-02-08")                
    },
    'unseen_appliances': {
        2: ("2013-06-01", "2013-06-07")
    }
}

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows=WINDOWS
    )
}



In [35]:
from neuralnilm.data.stridesource import StrideSource

In [36]:
stride_source = StrideSource(
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD,
    stride=STRIDE
)

INFO:neuralnilm.data.stridesource:Loading NILMTK data...
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_1...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:06+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_2...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_1...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.stridesource:Done loading NILMTK mains data.


In [37]:
series = stride_source._num_seqs['train'].cumsum()

In [38]:
for building_name, num_seq in series.iteritems():
    print(building_name, num_seq)

UK-DALE_building_1 1725


In [51]:
#stride_source._seq_i['train'] = 40
seq = stride_source.get_sequence()
seq.metadata

{'building_name': 'UK-DALE_building_1',
 'end_date': Timestamp('2014-01-01 18:46:24+0000', tz='Europe/London'),
 'seq_i': 43,
 'start_date': Timestamp('2014-01-01 18:20:54+0000', tz='Europe/London'),
 'total_num_sequences': 1725}

In [52]:
fig, axes = plt.subplots(2, sharex=True)
axes[0].plot(seq.target)
axes[1].plot(seq.input)
plt.show()

In [3]:
nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 111 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 119 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 30 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [4]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [5]:
from neuralnilm.data.realaggregatesource import RealAggregateSource

In [6]:
ras = RealAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD
)

INFO:neuralnilm.data.realaggregatesource:Loading NILMTK mains...
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:00+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_2...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Done loading NILMTK mains data.
INFO:neuralnilm.data.realaggregatesource:Found 89 sections without targ

In [7]:
ras.target_inclusion_prob = 0.5
for i in range(50):
    seq = ras.get_sequence()

In [8]:
fig, axes = plt.subplots(2)
axes[0].plot(seq.input)
axes[1].plot(seq.target)
plt.show()

In [9]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0]._metadata

['name', 'building', 'appliance', 'fold']

In [10]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False,
    sample_period=SAMPLE_PERIOD
)

In [11]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [12]:
#all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [13]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#fig.suptitle(FOLD)
#plt.show()

In [14]:
sample = source.get_batch(num_seq_per_batch=1024)

In [15]:
pipeline = DataPipeline(
    sources=[source, ras],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std()), IndependentlyCenter()],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [16]:
report = pipeline.report()
report

{'pipeline': {'input_processing': [{'divisor': 605.65094, 'name': 'DivideBy'},
   {'name': 'IndependentlyCenter'}],
  'num_seq_per_batch': 64,
  'rng_seed': None,
  'source_probabilities': [0.5, 0.5],
  'sources': {0: {'allow_incomplete_distractors': True,
    'allow_incomplete_target': False,
    'distractor_inclusion_prob': 0.25,
    'include_incomplete_target_in_output': True,
    'name': 'SyntheticAggregateSource',
    'rng_seed': None,
    'sample_period': 6,
    'seq_length': 256,
    'target_appliance': 'kettle',
    'target_inclusion_prob': 0.5,
    'uniform_prob_of_selecting_each_building': True},
   1: {'allow_incomplete_target': True,
    'allow_multiple_target_activations_in_aggregate': False,
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'include_incomplete_target_in_output': True,
    'include_multiple_targets_in_output': False,
    'name': 'RealAggregateSource',
    'rng_seed': None,
    'sample_period': 6,
    'seq_length': 256,
    'target_appliance': 'kett

In [17]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_2,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [18]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [19]:
from pymongo import MongoClient
client = MongoClient()
db = client.neuralnilm

In [20]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    experiment_id=["1"],
    metrics=Metrics(state_boundaries=[4]),
    learning_rates={0: 1E-2},
    repeat_callbacks=[
        (1000, Trainer.validate)
    ]
)

Database already has an experiment with _id == 0. Should the old experiment be deleted (both from the database and from disk)? Or quit? [Q/d] d


INFO:neuralnilm.trainer:Deleting documents for old experiment.
INFO:neuralnilm.trainer:Directory exists = '~/temp/neural_nilm/output/0'
INFO:neuralnilm.trainer:  Deleting directory.


In [21]:
report = trainer.report()
report['data']['activations'] = LOADER_CONFIG
from neuralnilm.utils import sanitise_dict_for_mongo


In [22]:
sanitise_dict_for_mongo(report)

{'_id': '0',
 'data': {'activations': {'nilmtk_activations': {'appliances': ['kettle',
     'microwave',
     'washing machine'],
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'sample_period': 6,
    'windows': {'train': {'1': ('2014-01-01', '2014-02-01')},
     'unseen_activations_of_seen_appliances': {'1': ('2014-02-02',
       '2014-02-08')},
     'unseen_appliances': {'2': ('2013-06-01', '2013-06-07')}}}},
  'pipeline': {'input_processing': [{'divisor': 605.6509399414062,
     'name': 'DivideBy'},
    {'name': 'IndependentlyCenter'}],
   'num_seq_per_batch': 64,
   'rng_seed': None,
   'source_probabilities': [0.5, 0.5],
   'sources': {'0': {'allow_incomplete_distractors': True,
     'allow_incomplete_target': False,
     'distractor_inclusion_prob': 0.25,
     'include_incomplete_target_in_output': True,
     'name': 'SyntheticAggregateSource',
     'rng_seed': None,
     'sample_period': 6,
     'seq_length': 256,
     'target_appliance': 'kettle',
     'target_inclus

In [23]:
trainer.fit(100000)

INFO:neuralnilm.trainer:Starting training for 100000 iterations.
INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.


   Update # |  Train cost  | Secs per update | Source ID
------------|--------------|-----------------|-----------
          0 |   1.151491  |    1.957127     |   0

INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.



          1 |   1.324135  |    0.056861     |   0
          2 |   1.045965  |    0.123030     |   1
          3 |   1.068326  |    0.069135     |   0
          4 |   1.130651  |    0.064761     |   0
          5 |   0.911296  |    0.280271     |   1
          6 |   1.023096  |    0.257015     |   1
          7 |   0.985004  |    0.224272     |   1
          8 |   1.179428  |    0.078045     |   0
          9 |   1.116361  |    0.239936     |   1
         10 |   1.178637  |    0.201006     |   1
         11 |   1.207232  |    0.093344     |   0
         12 |   1.024660  |    0.224247     |   1
         13 |   1.401876  |    0.127714     |   0
         14 |   1.110623  |    0.087405     |   0
         15 |   1.029661  |    0.131245     |   0
         16 |   1.061193  |    0.084805     |   0
         17 |   1.161034  |    0.186160     |   1
         18 |   1.069116  |    0.129243     |   0
         19 |   1.155509  |    0.237673     |   1
         20 |   1.144250  |    0.234837     |   1

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Keyboard interrupt at iteration 314.


...

INFO:neuralnilm.trainer:Starting training for 100000 iterations.
INFO:neuralnilm.trainer:Keyboard interrupt at iteration 641.


Please enter one or more letters: d
ipdb> self.learning_rate = 1E-4


INFO:neuralnilm.trainer:Iteration 641: Change learning rate to 1.0E-04


ipdb> c


INFO:neuralnilm.trainer:Starting training for 100000 iterations.
INFO:neuralnilm.trainer:Stopped training. Completed 100000 iterations.


In [24]:
net.train_iterations

100000

In [ ]:
from neuralnilm.monitor.monitor import Monitor

mon = Monitor("0")

In [ ]:
mon._plot_train_scores()

In [ ]:
mon._plot_validation_scores()

In [ ]:
mon.validation_metric_names

In [ ]:
mon.db.validation_scores.distinct(key='source_id')

In [ ]:
bool(trainer.db.train_scores.find_one({'experiment_id': trainer.experiment_id}))

In [ ]:
trainer.db.train_scores.delete_many?